In [42]:
%run UI.py

2.18.1
3.10.0


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\test2\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\qing5\Documents\Documents\JupyterScripts\Thesis\AIT2104019_Experiment\UI.py", line 14, in show_output
    hate_speech_value = prediction(preprocessed_input_string)
  File "C:\Users\qing5\Documents\Documents\JupyterScripts\Thesis\AIT2104019_Experiment\prediction.py", line 49, in prediction
  File "C:\Users\qing5\Documents\Documents\JupyterScripts\Thesis\AIT2104019_Experiment\prediction.py", line 33, in tokenization
    with open('posts1352_standard_GRU_tokenizer_2025_2.pickle', 'rb') as tokenizer_path:
ModuleNotFoundError: No module named 'keras.src.preprocessing'


In [5]:
import tensorflow as tf

import keras.layers as layer

In [7]:
import keras
print(keras.__version__)

3.10.0


In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [3]:
import spacy

In [9]:
from tensorflow.keras import preprocessing

In [7]:
import pickle

In [24]:
import string
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


import spacy

nlp = spacy.load('en_core_web_sm')

stopword_extra = ["always","want","even","still","ever","also","already","yet","basically","actually","need","please","ago","probable","probably","however","instead","quite","nt","na","u","gon","lol","im","ca","us","cnt","wo","em","etc","ll","aint","r","cant","shouldnt","wont","lah","dont","never"]

stopwords = nltk.corpus.stopwords.words('english')
stopwords += stopword_extra

negative_list = ['not','never','ain','aint','no','neither','nor','nt','cant','dont',"cnt",'wont',"shouldnt"]

def preprocess(input):
    preprocessed_input = []

    input = sent_tokenize(input)

    for text in input:

        #1. Generating the list of words in the tweet (hastags and other punctuations removed)
        text_blob = TextBlob(text)
        text = ' '.join(text_blob.words)

        # remove number
        text = re.sub(r'[0-9]', '', text)

        # lowercase
        text = text.lower()

        text = text.replace('/',' ')

        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')

        text = word_tokenize(text)

        #keep tokens that are alphabet characters
        text = [t for t in text if t.isalpha()]

        # replace the negation token
        replacer  = AntonymReplacer()
        text = replacer.replace_negations(text)

        # remove the stopwords
        text = [i for i in text if i not in stopwords]

        #remove empty token
        text = [t for t in text if len(t) > 0]

        preprocessed_input.append(text)


    return preprocessed_input


def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(" ".join(sent))
    texts_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) >= 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word in negative_list and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import keras

max_sequence_length = 46

def change_to_string(input):
    # obtain token list that's actually list instead of string
    from ast import literal_eval

    topic_words = input

    #for x in input:
    #    topic_words.append((literal_eval(x)))

    # change the list containing a list of elements into list of strings
    topic_strings = []

    for i in topic_words:
        temp = ' '.join(i)
        topic_strings.append(temp)

    topic_string = ' '.join(topic_strings)

    topic_string = [topic_string]

    return topic_string

def tokenization(input):
    
    # Load the tokenizer from the file
    with open('posts1352_standard_GRU_tokenizer_2025.pickle', 'rb') as tokenizer_path:
        loaded_tokenizer = pickle.load(tokenizer_path)

    sequences = loaded_tokenizer.texts_to_sequences(input)

    padded_data = pad_sequences(sequences, maxlen=max_sequence_length)

    return padded_data

def prediction(input):

    model = tf.keras.models.load_model('posts1352_standard_GRU_model_2025.h5')
    #model = tf.keras.models.load_model('posts1352_standard_GRU_model.h5',compile=False)
    #model.compile() 

    input = change_to_string(input)

    input = tokenization(input)

    predictions = model.predict(input)

    binary_predictions = (predictions > 0.5).astype(int)

    return binary_predictions[0][0]

In [40]:
input_string = "not all the malay people their education is good"

preprocessed_input_string = preprocess(input_string)
hate_speech_value = prediction(preprocessed_input_string)
hate_speech_value

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step


0

In [39]:
model = tf.keras.models.load_model('posts1352_standard_GRU_model.h5')

C:\ProgramData\Anaconda3\envs\test2\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\test2\lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Unrecognized keyword arguments passed to GRU: {'time_major': False}